## Set-Up the Environment

In [ ]:
import pyabf
import slopeTools
import abfTools
import plotTools
import statsTools
import numpy as np
import matplotlib.pyplot as plt

## Moving Window Mean

This function creates a new set of data where each point is the mean of the adjacent `N` points (where `N` is defined by `windowSize`)

In [ ]:
def demonstrateMovingWindowMean(pointCount = 100):
    """
    Demonstrate the moving window average method on random data using multiple window sizes
    """
    values = np.cumsum(np.random.random_sample(pointCount) - .5) # random walk
    times = np.arange(pointCount) * 10/60

    plt.figure()
    plt.plot(times, values, label="original")

    for windowSize in [5, 10, 15]:
        smoothValues, smoothTimes = statsTools.smoothY(values, times, windowSize) 
        plt.plot(smoothTimes, smoothValues, label=f"window: {windowSize}")
    
    plt.title("Moving Window Average")
    plt.legend()
    plt.show()
    
demonstrateMovingWindowMean()

## ABF Holding Current Slope Analysis
* Get holding current from every sweep
* Apply a moving window filter to smooth the data
* Report the slope between every data point

In [ ]:
windowSize = 15 #sweeps
sweepLength = 10/60 #min
abfIDs = []
currentsBaseline = []
currentsDrug = []
meanSlopesBaseline = []
peakSlopesDrug = []

abfFilePaths = [
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804007.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804024.abf", 
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804030.abf", 
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804043.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804048.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804060.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804066.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20805008.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20805029.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20805035.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20811011.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20811021.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20817012.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20831011.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20831017.abf"
]

for abfFilePath in abfFilePaths:
    
    # Get raw holding current
    currentsRaw, timesRaw, abfID = abfTools.meanIhold(abfFilePath)
    abfIDs.append(abfID)
    
    # Get the mean current during baseline and drug application
    currentsBaseline.append(statsTools.rangeMean(currentsRaw, timesRaw, 3, 5))
    currentsDrug.append(statsTools.rangeMean(currentsRaw, timesRaw, 6, 10))
    
    # Get smoothed holding current
    currentsSmooth, currentsSmoothTimes = statsTools.smoothY(currentsRaw, timesRaw, windowSize)
    
    # Get slopes of consecutive data points
    slopes = slopeTools.consecutiveSlopes(currentsSmooth, currentsSmoothTimes)
    slopeTimes = currentsSmoothTimes[0:len(slopes)]
        
    # Determine the time of the first tag. Later it will be aligned to be 5 minutes
    firstTagTime = abfTools.getFirstTagTime(abfFilePath)

    # Get the peak slope during a limited range of time
    peakSlopeTimeMin = 5
    peakSlopeTimeMax = 10
    peakSlope = statsTools.rangeMin(slopes, currentsSmoothTimes, peakSlopeTimeMin, peakSlopeTimeMax)
    peakSlopesDrug.append(peakSlope)
    peakSlopeIndex = slopes.index(peakSlope)
    peakSlopeTime = currentsSmoothTimes[peakSlopeIndex]
    
    # Get the mean slope during baseline (3-5 min)
    meanSlopeBaseline = statsTools.rangeMean(slopes, currentsSmoothTimes, 3, 5)
    meanSlopesBaseline.append(meanSlopeBaseline)
    
    # Plot the data and its slopes

    
    plt.figure()
    ax1 = plt.subplot(211)
    plotTools.plotCurrentOverTimeAndLabelPeakSlope(currentsRaw, timesRaw, peakSlopeTime, windowSize)
    plt.title(abfID)

    ax2 = plt.subplot(212, sharex=ax1)
    plotTools.plotSlopesOverTimeAndMarkpeakSlope(slopes, slopeTimes, peakSlopeTime, peakSlope)
    
    plt.show()
    

### Plot slopes before and during drug application

In [ ]:
plotTools.plotPairs(meanSlopesBaseline, peakSlopesDrug, "Slopes")
plotTools.plotPairs(currentsBaseline, currentsDrug, "Mean Current")

### Report responders and non-responders based on the given criterias
1. Change of slope is larger than the defined threshold.
2. The delta holding current during drug application period is less than 0.

_Slopes are in pA/min_

In [ ]:
slopeThreshold = -1.5 # peak negative slope must exceed this value to be a responder

# create lists of responders and non-responders
responders=[]
nonResponders=[]
for i in range(len(abfIDs)):
    abfID = abfIDs[i]
    drugEffectSlope = peakSlopesDrug[i] - meanSlopesBaseline[i]
    cellHasSteepNegativeSlope = drugEffectSlope <= slopeThreshold
    cellGoesNegativeOverTime = currentsDrug[i]-currentsBaseline[i] < 0
    if cellHasSteepNegativeSlope and cellGoesNegativeOverTime:
        responders.append(abfID)
    else:
        nonResponders.append(abfID)

responseRate = round(len(responders)/len(abfIDs)*100, 3)
print(f"Responders = {len(responders)}")
print(f"Non-Responders = {len(nonResponders)}")
print(f"Response rate = {responseRate}%")

In [ ]:
print("Responders: " + ", ".join(responders))
print("Non-Responders: " + ", ".join(nonResponders))